In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import TruncatedSVD
import cluster_class
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import train_test_split
import keras.utils

unable to import 'smart_open.gcs', disabling that module
Using TensorFlow backend.


In [2]:
f = open("../clean_text_with_punc.pickle","rb")
df = pickle.load(f)
f.close()
df.head()

,sentence,author
0,"[eugenie, grandet, by, honore, de, balzac, tra...","Balzac, Honoré de"
1,"[may, your, name,, that, of, one, whose, portr...","Balzac, Honoré de"
2,"[de, balzac.]","Balzac, Honoré de"
3,"[eugenie, grandet, i, there, are, houses, in, ...","Balzac, Honoré de"
4,"[within, these, houses, there, is,, perhaps,, ...","Balzac, Honoré de"


In [3]:
# turn list into string
df['sentence'] = df['sentence'].apply(lambda x:' '.join(x))
df.head()

,sentence,author
0,eugenie grandet by honore de balzac translated...,"Balzac, Honoré de"
1,"may your name, that of one whose portrait is t...","Balzac, Honoré de"
2,de balzac.,"Balzac, Honoré de"
3,eugenie grandet i there are houses in certain ...,"Balzac, Honoré de"
4,"within these houses there is, perhaps, the sil...","Balzac, Honoré de"


In [4]:
# turn author into tag
#[0,0,0,1,0,0,0,0,.....]
# author_list = list(set(df['author']))
# label = []
# for i in df['author']:
#     label.append([1 if author_list[j] == i else 0 for j in range(len(author_list))])

# [0,15,13,7,3,...]
author_list = list(set(df['author']))
label = []
for i in df['author']:
    label.append(author_list.index(i))

In [5]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['sentence'].values)]

In [8]:
model = Doc2Vec(documents, vector_size=10, window=2, min_count=1, workers=4)

# model = Word2Vec(lines, size=20, window=3, min_count=1, workers=4, iter=10)
# model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

# todo: 
# train, test = train_test_split(examples)
documents[:5]

[TaggedDocument(words='eugenie grandet by honore de balzac translated by katharine prescott wormeley dedication to maria.', tags=[0]),
 TaggedDocument(words='may your name, that of one whose portrait is the noblest ornament of this work, lie on its opening pages like a branch of sacred box, taken from an unknown tree, but sanctified by religion, and kept ever fresh and green by pious hands to bless the house.', tags=[1]),
 TaggedDocument(words='de balzac.', tags=[2]),
 TaggedDocument(words='eugenie grandet i there are houses in certain provincial towns whose aspect inspires melancholy, akin to that called forth by sombre cloisters, dreary moorlands, or the desolation of ruins.', tags=[3]),
 TaggedDocument(words='within these houses there is, perhaps, the silence of the cloister, the barrenness of moors, the skeleton of ruins; life and movement are so stagnant there that a stranger might think them uninhabited, were it not that he encounters suddenly the pale, cold glance of a motionles

In [9]:
f = open("word_model_Doc2Vec.pickle","wb")
pickle.dump(model, f)
f.close()

In [10]:
X = []
for i in df['sentence']:
    X.append(model.infer_vector(i.split()))
X[0]

array([-0.00971177, -0.00748824, -0.00461749,  0.00079337, -0.01550109,
       -0.0349376 , -0.03031154,  0.01179206,  0.00804681,  0.02876098],
      dtype=float32)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.2, random_state=42)

In [14]:
lg = LogisticRegression(solver='saga',max_iter=500, multi_class='multinomial').fit(X_train, y_train)


In [21]:
right, wrong = 0, 0
for i, j in zip(lg.predict(X_test), y_test):
    if i == j:
        right += 1
    else:
        wrong += 1
right/wrong # accuracy

0.10996835904039903